<a href="https://colab.research.google.com/github/Nov05/DS-Unit-2-Sprint-4-Project/blob/master/notebooks/02_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# created by nov05 on 2019-08-01

plotly example  https://colab.research.google.com/drive/1oijny-_J2K-7ma6jNq9jAp7C_tzNB27W  
https://medium.com/@plotlygraphs/plotly-py-4-0-is-here-offline-only-express-first-displayable-anywhere-fc444e5659ee  
https://community.plot.ly/t/introducing-plotly-py-4-0-0/25639  

https://plot.ly/python/figure-factory-subplots/

In [0]:
# !pip uninstall plotly
# !pip install plotly==3.6.1
# !pip install plotly==4.0.0

In [0]:
import plotly

# import plotly.plotly as py
## ImportError: 
## The plotly.plotly module is deprecated,
## please install the chart-studio package and use the
## chart_studio.plotly module instead. 

import plotly.graph_objs as go
# plot = online mode, iplot = offline mode
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.graph_objs import Histogram2dContour, Scatter
from plotly.graph_objs.histogram2dcontour import Contours
from plotly.graph_objs.scatter import Marker

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [4]:
print("plotly", plotly.__version__)
# make sure it is version 4.0.0

plotly 4.0.0


In [0]:
# define this function for plotly and plotly express displaying
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
//// both would work, enjoy plotly and plotly express in colab
//               plotly: 'https://cdn.plot.ly/plotly-1.5.1.min.js?noext',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
# init_notebook_mode(connected=True)

# then call the function configure_plotly_browser_state() 
# in EVERY code block where there is 
# a Plotly plot (each will be an iframe) 

# import and merge datasets

In [0]:
url_tasks = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/Sip-task-info.csv"
url_dates = "https://raw.githubusercontent.com/Derek-Jones/SiP_dataset/master/est-act-dates.csv"
tasks  = pd.read_csv(url_tasks, encoding='iso-8859-1')
dates  = pd.read_csv(url_dates, encoding='iso-8859-1')

# merge two datasets
# there are duplicate TaskNumbers in both datasets
# there are duplicate rows in "dates" dataset
alldata = tasks.merge(right=dates[-dates.duplicated()], on='TaskNumber', how='inner')

breakdown = (alldata[['ProjectCode', 'ProjectBreakdownCode']]
             .groupby('ProjectCode').nunique()
             .drop('ProjectCode', axis=1)
             .reset_index())

def wrangler(X):
## data type conversion  
  X['EstimateOn'] = pd.to_datetime(X['EstimateOn'], format='%d-%b-%y')
  X['StartedOn'] = pd.to_datetime(X['StartedOn'], format='%d-%b-%y')
  X['CompletedOn'] = pd.to_datetime(X['CompletedOn'], format='%d-%b-%y')

## feature engineering
  X['hoursestimatelog'] = np.log1p(X['HoursEstimate'])
  X['daysactual'] = (X['CompletedOn']-X['StartedOn']).dt.days.abs()
  X['estimateonsameday'] = X['EstimateOn']==X['StartedOn']
  ## pandas.DataFrame.merge() behaves awkwardly in a function
  ## so I have to do it in this way
  tmp = (X[['ProjectCode']]
         .merge(right=breakdown, on='ProjectCode', how='left'))
  X['breakdown'] = tmp['ProjectBreakdownCode']
#   tmp = (X[['DeveloperID']].merge(right=performancelevel,
#                                   on='DeveloperID', how='left'))
#   X['performancelevel'] = tmp['performancelevel'].astype(int)

## missing values
  X.replace([np.inf, -np.inf, pd.NaT], np.nan, inplace=True)
wrangler(alldata)

In [7]:
print(alldata.columns.to_list())

['TaskNumber', 'Summary', 'Priority', 'RaisedByID', 'AssignedToID', 'AuthorisedByID', 'StatusCode', 'ProjectCode', 'ProjectBreakdownCode', 'Category', 'SubCategory', 'HoursEstimate', 'HoursActual', 'DeveloperID', 'DeveloperHoursActual', 'TaskPerformance', 'DeveloperPerformance', 'EstimateOn', 'StartedOn', 'CompletedOn', 'hoursestimatelog', 'daysactual', 'estimateonsameday', 'breakdown']


In [0]:
tbl_data = alldata[['Category', 'HoursEstimate', 'HoursActual', 'daysactual']]

In [9]:
configure_plotly_browser_state()
hist_data = [alldata['HoursActual'], alldata['HoursEstimate']]
group_labels = ['Hours Actual', 'Hours Estimated']
fig = ff.create_distplot(hist_data, group_labels, 
                         bin_size=.2, show_rug=False
                        )
# plotly 3.6.1: AttributeError: 'Figure' object has no attribute 'show'
# plotly 4.0.0: offline only
fig.show()